In [14]:

# Import the required libraries
import re
import httpx
from numpy import full
from selectolax.parser import HTMLParser
from dataclasses import dataclass, asdict, fields
import csv
from urllib.parse import urljoin

site_url = "https://scotlandsplaces.gov.uk"

main_url = "https://scotlandsplaces.gov.uk/digital-volumes/ordnance-survey-name-books/lanarkshire-os-name-books-1858-1861"

# Set the base url
base_url = "https://scotlandsplaces.gov.uk/digital-volumes/ordnance-survey-name-books/lanarkshire-os-name-books-1858-1861/lanarkshire-volume-01/"


# Create a dataclass to store the information
@dataclass
class placename_information:
    page_title: str | None
    placename: str = "No table on page"
    various_spellings: str = "No table on page"
    authority: str = "No table on page"
    situation: str = "No table on page"
    description: str = "No table on page"

def get_volumes(url):
    response = httpx.get(url)
    html = HTMLParser(response.text)
    links = html.css("a")

    volume_numbers = []
    for link in links:
        href = link.attributes.get("href")
        if href:
            match = re.search(r"/digital-volumes/ordnance-survey-name-books/lanarkshire-os-name-books-1858-1861/lanarkshire-volume-\d+", href)
            if match:
                full_url = urljoin(site_url, match.group())
                volume_numbers.append(full_url + "/")
    return list(set(volume_numbers))

# Create a function to get the page numbers
def get_page_numbers(url):
    response = httpx.get(url)
    html = HTMLParser(response.text)
    links = html.css("a")

    page_numbers = []
    for link in links:
        href = link.attributes.get("href")
        if href:
            match = re.search(r"\d+$", href)
            if match:
                page_numbers.append(int(match.group()))

    return list(set(page_numbers))


# Create a function to get the html
def get_html(page):
    url = f"{base_url}{page}"
    response = httpx.get(url)
    return HTMLParser(response.text)


# Create a function to parse the placename information
def parse_placename(html):
    page_header = html.css_first("h1.page-header").text()
    name = html.css("div.well tr")  # get each table row from table
    results = []
    for item in name:
        item_parts = item.css("td")
        if len(item_parts) == 5:  # this will avoid the table headers
            new_item = placename_information(
                page_title=page_header,
                placename=item_parts[0].text().strip(),
                various_spellings=item_parts[1].text().replace("\n", "|").strip(),
                authority=item_parts[2].text().replace("\n", "|").strip(),
                situation=item_parts[3].text().strip(),
                description=item_parts[4].text().strip(),
            )
            results.append(asdict(new_item))
    return results


# Create a function to write the results to a csv
def to_csv(res):
    with open("name_books.csv", "a", newline="") as file:
        fieldnames = [field.name for field in fields(placename_information)]
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        if file.tell() == 0:
            writer.writeheader()
        writer.writerows(res)


# Create a main function to run the code
def main():
    url = base_url
    volumes = get_volumes(main_url)
    print(volumes)
    # page_numbers = get_page_numbers(url)
    # for page in page_numbers:
    #     html = get_html(page)
    #     print(html.css_first("title").text())
    #     results = parse_placename(html)
    #     print(results)
    #     to_csv(results)


if __name__ == "__main__":
    main()

['https://scotlandsplaces.gov.uk/digital-volumes/ordnance-survey-name-books/lanarkshire-os-name-books-1858-1861/lanarkshire-volume-57/', 'https://scotlandsplaces.gov.uk/digital-volumes/ordnance-survey-name-books/lanarkshire-os-name-books-1858-1861/lanarkshire-volume-40/', 'https://scotlandsplaces.gov.uk/digital-volumes/ordnance-survey-name-books/lanarkshire-os-name-books-1858-1861/lanarkshire-volume-04/', 'https://scotlandsplaces.gov.uk/digital-volumes/ordnance-survey-name-books/lanarkshire-os-name-books-1858-1861/lanarkshire-volume-12/', 'https://scotlandsplaces.gov.uk/digital-volumes/ordnance-survey-name-books/lanarkshire-os-name-books-1858-1861/lanarkshire-volume-08/', 'https://scotlandsplaces.gov.uk/digital-volumes/ordnance-survey-name-books/lanarkshire-os-name-books-1858-1861/lanarkshire-volume-15/', 'https://scotlandsplaces.gov.uk/digital-volumes/ordnance-survey-name-books/lanarkshire-os-name-books-1858-1861/lanarkshire-volume-05/', 'https://scotlandsplaces.gov.uk/digital-volumes